In [18]:
import pandas as pd
import numpy as np
import os.path
import requests
import json

root_path = os.path.dirname(os.getcwd())

# Import food inspection data
inspections = pd.read_csv(os.path.join(root_path, "DATA/food_inspections.csv"))

In [19]:
# Create weather dataframe with entry for each inspection
weather = pd.DataFrame({
    "inspection_id": inspections.inspection_id,
    "date": inspections.inspection_date,
})

# Create list of weather data columns
features = [
    "precip_intensity",
    "temperature_max",
    "wind_speed",
    "humidity"
]

# Add empty columns
weather = pd.concat([weather, pd.DataFrame(columns=features)])

In [20]:
# Check if weather data exists, import or start empty dataframe
if os.path.isfile(os.path.join(root_path, "DATA/weather.csv")):
    old_weather = pd.read_csv(os.path.join(root_path, "DATA/weather.csv"))
else:
    old_weather = pd.DataFrame(columns=weather.columns)

In [21]:
# Register for a free key at https://darksky.net/dev/register
KEY = "c30a95e4e9f7f2e90966361288918b9e"

In [22]:
# Check if weather record can be found in old data
# If not,request record from darksky.net
# All records pertain to a single location in central Chicago

limit_reached = False

def get_weather(group):
    global limit_reached
    
    date = group.iloc[0].date
    record = old_weather[old_weather.date == date]
    if (not record.empty) & (not record.isnull().values.any()):
        print(date, "record found")
        group.loc[:, features] = record.iloc[0][features].values
        
    elif not limit_reached:
        url = "https://api.darksky.net/forecast/%s/41.836944,-87.684722,%s" % (KEY, date)
        response = requests.get(url)
        
        if response.status_code == 200:
            print(date, "request successful")
            data = response.json()["daily"]["data"][0] 
            group[features] = [
                data["precipIntensity"],
                data["temperatureMax"],
                data["windSpeed"],
                data["humidity"]
            ]
        elif response.status_code == 403:
            print("request limit reached")
            limit_reached = True
        else:
            print(response)
    return group

In [23]:
# Time consuming
weather = weather.groupby(by="date").apply(get_weather)

2010-01-04T00:00:00 record found
2010-01-04T00:00:00 record found
2010-01-05T00:00:00 record found
2010-01-06T00:00:00 record found
2010-01-07T00:00:00 record found
2010-01-08T00:00:00 record found
2010-01-11T00:00:00 record found
2010-01-12T00:00:00 record found
2010-01-13T00:00:00 record found
2010-01-14T00:00:00 record found
2010-01-15T00:00:00 record found
2010-01-19T00:00:00 record found
2010-01-20T00:00:00 record found
2010-01-21T00:00:00 record found
2010-01-22T00:00:00 record found
2010-01-25T00:00:00 record found
2010-01-26T00:00:00 record found
2010-01-27T00:00:00 record found
2010-01-28T00:00:00 record found
2010-02-01T00:00:00 record found
2010-02-02T00:00:00 record found
2010-02-03T00:00:00 record found
2010-02-04T00:00:00 record found
2010-02-05T00:00:00 record found
2010-02-08T00:00:00 record found
2010-02-09T00:00:00 record found
2010-02-10T00:00:00 record found
2010-02-11T00:00:00 record found
2010-02-16T00:00:00 record found
2010-02-17T00:00:00 record found
2010-02-18

2011-01-28T00:00:00 record found
2011-01-31T00:00:00 record found
2011-02-01T00:00:00 record found
2011-02-03T00:00:00 record found
2011-02-04T00:00:00 record found
2011-02-07T00:00:00 record found
2011-02-08T00:00:00 record found
2011-02-09T00:00:00 record found
2011-02-10T00:00:00 record found
2011-02-14T00:00:00 record found
2011-02-15T00:00:00 record found
2011-02-16T00:00:00 record found
2011-02-17T00:00:00 record found
2011-02-18T00:00:00 record found
2011-02-22T00:00:00 record found
2011-02-23T00:00:00 record found
2011-02-24T00:00:00 record found
2011-02-25T00:00:00 record found
2011-02-28T00:00:00 record found
2011-03-01T00:00:00 record found
2011-03-02T00:00:00 record found
2011-03-03T00:00:00 record found
2011-03-04T00:00:00 record found
2011-03-08T00:00:00 record found
2011-03-09T00:00:00 record found
2011-03-10T00:00:00 record found
2011-03-11T00:00:00 record found
2011-03-14T00:00:00 record found
2011-03-15T00:00:00 record found
2011-03-16T00:00:00 record found
2011-03-17

2012-02-15T00:00:00 record found
2012-02-16T00:00:00 record found
2012-02-17T00:00:00 record found
2012-02-21T00:00:00 record found
2012-02-24T00:00:00 record found
2012-02-27T00:00:00 record found
2012-02-28T00:00:00 record found
2012-02-29T00:00:00 record found
2012-03-01T00:00:00 record found
2012-03-02T00:00:00 record found
2012-03-03T00:00:00 record found
2012-03-06T00:00:00 record found
2012-03-07T00:00:00 record found
2012-03-08T00:00:00 record found
2012-03-09T00:00:00 record found
2012-03-12T00:00:00 record found
2012-03-13T00:00:00 record found
2012-03-14T00:00:00 record found
2012-03-15T00:00:00 record found
2012-03-16T00:00:00 record found
2012-03-19T00:00:00 record found
2012-03-20T00:00:00 record found
2012-03-21T00:00:00 record found
2012-03-22T00:00:00 record found
2012-03-26T00:00:00 record found
2012-03-27T00:00:00 record found
2012-03-28T00:00:00 record found
2012-03-29T00:00:00 record found
2012-03-30T00:00:00 record found
2012-04-02T00:00:00 record found
2012-04-03

2013-03-14T00:00:00 record found
2013-03-15T00:00:00 record found
2013-03-18T00:00:00 record found
2013-03-19T00:00:00 record found
2013-03-20T00:00:00 record found
2013-03-21T00:00:00 record found
2013-03-22T00:00:00 record found
2013-03-25T00:00:00 record found
2013-03-26T00:00:00 record found
2013-03-27T00:00:00 record found
2013-03-28T00:00:00 record found
2013-03-29T00:00:00 record found
2013-04-01T00:00:00 record found
2013-04-02T00:00:00 record found
2013-04-03T00:00:00 record found
2013-04-04T00:00:00 record found
2013-04-05T00:00:00 record found
2013-04-08T00:00:00 record found
2013-04-09T00:00:00 record found
2013-04-10T00:00:00 record found
2013-04-11T00:00:00 record found
2013-04-12T00:00:00 record found
2013-04-15T00:00:00 record found
2013-04-16T00:00:00 record found
2013-04-17T00:00:00 record found
2013-04-18T00:00:00 record found
2013-04-19T00:00:00 record found
2013-04-22T00:00:00 record found
2013-04-23T00:00:00 record found
2013-04-24T00:00:00 record found
2013-04-25

2014-03-27T00:00:00 record found
2014-03-28T00:00:00 record found
2014-03-31T00:00:00 record found
2014-04-01T00:00:00 record found
2014-04-02T00:00:00 record found
2014-04-03T00:00:00 record found
2014-04-04T00:00:00 record found
2014-04-07T00:00:00 record found
2014-04-08T00:00:00 record found
2014-04-09T00:00:00 record found
2014-04-10T00:00:00 record found
2014-04-11T00:00:00 record found
2014-04-14T00:00:00 record found
2014-04-15T00:00:00 record found
2014-04-16T00:00:00 record found
2014-04-17T00:00:00 record found
2014-04-18T00:00:00 record found
2014-04-21T00:00:00 record found
2014-04-22T00:00:00 record found
2014-04-23T00:00:00 record found
2014-04-24T00:00:00 record found
2014-04-25T00:00:00 record found
2014-04-28T00:00:00 record found
2014-04-29T00:00:00 record found
2014-04-30T00:00:00 record found
2014-05-01T00:00:00 record found
2014-05-02T00:00:00 record found
2014-05-05T00:00:00 record found
2014-05-06T00:00:00 record found
2014-05-07T00:00:00 record found
2014-05-08

2015-04-07T00:00:00 record found
2015-04-08T00:00:00 record found
2015-04-09T00:00:00 record found
2015-04-10T00:00:00 record found
2015-04-13T00:00:00 record found
2015-04-14T00:00:00 record found
2015-04-15T00:00:00 record found
2015-04-16T00:00:00 record found
2015-04-17T00:00:00 record found
2015-04-20T00:00:00 record found
2015-04-21T00:00:00 record found
2015-04-22T00:00:00 record found
2015-04-23T00:00:00 record found
2015-04-24T00:00:00 record found
2015-04-27T00:00:00 record found
2015-04-28T00:00:00 record found
2015-04-29T00:00:00 record found
2015-04-30T00:00:00 record found
2015-05-01T00:00:00 record found
2015-05-05T00:00:00 record found
2015-05-06T00:00:00 record found
2015-05-07T00:00:00 record found
2015-05-08T00:00:00 record found
2015-05-11T00:00:00 record found
2015-05-12T00:00:00 record found
2015-05-13T00:00:00 record found
2015-05-14T00:00:00 record found
2015-05-15T00:00:00 record found
2015-05-18T00:00:00 record found
2015-05-19T00:00:00 record found
2015-05-20

2016-04-20T00:00:00 record found
2016-04-21T00:00:00 record found
2016-04-22T00:00:00 record found
2016-04-25T00:00:00 record found
2016-04-26T00:00:00 record found
2016-04-27T00:00:00 record found
2016-04-28T00:00:00 record found
2016-04-29T00:00:00 record found
2016-05-02T00:00:00 record found
2016-05-03T00:00:00 record found
2016-05-04T00:00:00 record found
2016-05-05T00:00:00 record found
2016-05-06T00:00:00 record found
2016-05-09T00:00:00 record found
2016-05-10T00:00:00 record found
2016-05-11T00:00:00 record found
2016-05-12T00:00:00 record found
2016-05-13T00:00:00 record found
2016-05-16T00:00:00 record found
2016-05-17T00:00:00 record found
2016-05-18T00:00:00 record found
2016-05-19T00:00:00 record found
2016-05-20T00:00:00 record found
2016-05-23T00:00:00 record found
2016-05-24T00:00:00 record found
2016-05-25T00:00:00 record found
2016-05-26T00:00:00 record found
2016-05-27T00:00:00 record found
2016-05-31T00:00:00 record found
2016-06-01T00:00:00 record found
2016-06-02

2017-05-16T00:00:00 record found
2017-05-17T00:00:00 record found
2017-05-18T00:00:00 record found
2017-05-19T00:00:00 record found
2017-05-22T00:00:00 record found
2017-05-23T00:00:00 record found
2017-05-24T00:00:00 record found
2017-05-25T00:00:00 record found
2017-05-26T00:00:00 record found
2017-05-30T00:00:00 record found
2017-05-31T00:00:00 record found
2017-06-01T00:00:00 record found
2017-06-02T00:00:00 record found
2017-06-05T00:00:00 record found
2017-06-06T00:00:00 record found
2017-06-07T00:00:00 record found
2017-06-08T00:00:00 record found
2017-06-09T00:00:00 record found
2017-06-12T00:00:00 record found
2017-06-13T00:00:00 record found
2017-06-14T00:00:00 record found
2017-06-15T00:00:00 record found
2017-06-16T00:00:00 record found
2017-06-19T00:00:00 record found
2017-06-20T00:00:00 record found
2017-06-21T00:00:00 record found
2017-06-22T00:00:00 record found
2017-06-23T00:00:00 record found
2017-06-26T00:00:00 record found
2017-06-27T00:00:00 record found
2017-06-28

2018-05-15T00:00:00 request successful
2018-05-16T00:00:00 request successful
2018-05-17T00:00:00 request successful
2018-05-18T00:00:00 request successful
2018-05-21T00:00:00 request successful
2018-05-22T00:00:00 request successful
2018-05-23T00:00:00 request successful
2018-05-24T00:00:00 request successful
2018-05-25T00:00:00 request successful
2018-05-29T00:00:00 request successful
2018-05-30T00:00:00 request successful
2018-05-31T00:00:00 request successful
2018-06-01T00:00:00 request successful
2018-06-04T00:00:00 request successful
2018-06-05T00:00:00 request successful
2018-06-06T00:00:00 request successful
2018-06-07T00:00:00 request successful
2018-06-08T00:00:00 request successful
2018-06-11T00:00:00 request successful
2018-06-12T00:00:00 request successful
2018-06-13T00:00:00 record found
2018-06-14T00:00:00 record found
2018-06-15T00:00:00 record found
2018-06-19T00:00:00 record found


In [24]:
# Check if process is complete
if weather.isnull().values.any():
    print("Process incomplete, records missing")
else:
    print("Process complete, all records obtained")

Process complete, all records obtained


Darksky.net limits free accounts to 1000 calls per day. If records are missing, save the result and run the notebook from the top after waiting 24 hours, upgrading your account or obtaining a new API key.

In [26]:
# Save result
weather.to_csv(os.path.join(root_path, "DATA/weather.csv"), index=False)